In [1]:
%pip install numpy --upgrade

#"tensorflow-macos>=2.10.0",
#"tensorflow-metal==0.6.0"

  Using cached numpy-1.24.3-cp39-cp39-macosx_10_9_x86_64.whl (19.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [51]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense

In [52]:
df = pd.read_csv("dining/synthetic_transaction_data_Dining.csv")

df.head()

,transaction_id,transaction_date,cardholder_name,card_number,card_type,merchant_name,merchant_category,merchant_state,merchant_city,transaction_status,transaction_amount,merchant_category_code,fraud_flag
0,PJ82FF-U65G-T9SH9V,2018-09-18 23:40:00,Kristin Williams,4032380333074516,visa,KFC,Fast Food,New Jersey,Jersey City,Declined,5.397294,MCC 5814,0
1,5WFUH5-VMPB-RHOXKV,2020-01-17 16:30:00,Andrea Melendez,4736903953786546,visa,McDonald's,Fast Food,Montana,Missoula,Successful,27.993661,MCC 5814,0
2,TW1LD6-3CGZ-MQW4KN,2022-08-26 22:41:00,Katie Woods,4534834729087782,visa,Domino's Pizza,Fast Food,Ohio,Cleveland,Pending,26.724409,MCC 5814,0
3,DKKV7Q-H2LA-406TWY,2018-04-18 20:25:00,Sherry Palmer,4683926616868339,visa,McDonald's,Fast Food,Massachusetts,Worcester,Declined,26.898345,MCC 5814,0
4,0CK2EQ-K275-9AC5UT,2023-05-26 21:55:00,Matthew Blake,4078169149501845,visa,Papa John's,Fast Food,Minnesota,Saint Paul,Successful,45.549538,MCC 5814,0


In [53]:
df_og = df[df['merchant_name'] == "Cheesecake Factory"].copy().reset_index()
del df_og['index']
df_og.head()

,transaction_id,transaction_date,cardholder_name,card_number,card_type,merchant_name,merchant_category,merchant_state,merchant_city,transaction_status,transaction_amount,merchant_category_code,fraud_flag
0,GMHMTR-LG4Z-FBR2HQ,2023-07-06 20:49:00,Victor Carr,4179994364612649,visa,Cheesecake Factory,Casual Dining,North Carolina,Charlotte,Pending,2.654415,MCC 5812,0
1,QGGRBJ-IYNU-M066H9,2021-11-04 21:45:00,David Boyd,4020743055298296,visa,Cheesecake Factory,Casual Dining,Wisconsin,Madison,Successful,0.213500,MCC 5812,0
2,JU5E74-1UYB-H1IJBH,2019-05-26 22:10:00,Rhonda Anderson,4876969614410479,visa,Cheesecake Factory,Casual Dining,Hawaii,Honolulu,Declined,23.035066,MCC 5812,0
3,IVO5JG-82UL-FUKRWH,2022-06-11 08:07:00,Adrian Pacheco,4304190228039198,visa,Cheesecake Factory,Casual Dining,Michigan,Detroit,Declined,43.142440,MCC 5812,0
4,8VZVZR-AN3T-Z1IL4M,2019-05-24 08:23:00,Colin Cole,4397990976519202,visa,Cheesecake Factory,Casual Dining,Wyoming,Cheyenne,Declined,1.475463,MCC 5812,0


In [54]:
z_scores = np.abs((df_og['transaction_amount'] - df_og['transaction_amount'].mean()) / df_og['transaction_amount'].std())

# Define a threshold (e.g., Z-score > 3) to identify outliers
threshold = 3

# Remove outliers from the DataFrame
df_og = df_og[z_scores < threshold]

In [55]:
# create month-year column
df_og['transaction_date'] = pd.to_datetime(df_og['transaction_date'])
df_og['month'] = df_og['transaction_date'].dt.month
df_og = df_og[df_og['month'] <= 12]
df_og = df_og[df_og['month'] >= 1]
df_og['month'] = df_og['month'].astype(str)

df_og['year'] = df_og['transaction_date'].dt.year
df_og['year'] = df_og['year'].astype(str)

df_og['month_year'] =  df_og['year'] + "-"+  df_og['month'] 
df_og['month_year'] = pd.to_datetime(df_og['month_year'])

df_grouped = df_og.groupby('month_year')['transaction_amount'].sum().reset_index()
df_grouped = df_grouped.set_index('month_year').sort_index()
df_grouped.index = pd.to_datetime(df_grouped.index)

In [56]:
cutoff_point1 = pd.to_datetime("2023-03-01")
cutoff_point2 = pd.to_datetime("2023-06-01")
train_data = df_grouped[df_grouped.index < cutoff_point1]
test_data = df_grouped[(df_grouped.index >= cutoff_point1) & (df_grouped.index <= cutoff_point2)]

In [57]:
import plotly.graph_objects as go

# Create a trace for the train data
trace_train = go.Scatter(
    x=train_data.index,
    y=train_data['transaction_amount'],
    name='Train Data'
)

# Create a trace for the test data
trace_test = go.Scatter(
    x=test_data.index,
    y=test_data['transaction_amount'],
    name='Test Data'
)

# Create the layout for the plot
layout = go.Layout(
    title='Train and Test Data',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Transaction Amount')
)

# Create the figure object and add the traces to it
fig = go.Figure(data=[trace_train, trace_test], layout=layout)

# Show the plot
fig.show()

In [58]:

# Create the MinMaxScaler object
scaler = MinMaxScaler()

# Perform MinMax scalinon the training data
train_scaled = scaler.fit_transform(train_data)

# Perform MinMax scaling on the testing data
test_scaled = scaler.transform(test_data)


# Prepare the training data
X_train = np.array(train_scaled[:-1])  # Input sequence
y_train = np.array(train_scaled[1:])   # Target sequence

X_test = np.array(test_scaled[:-1])  # Input sequence
y_test = np.array(test_scaled[1:])   # Target sequence


In [59]:
# Build the GRU model
gru_model = Sequential()
gru_model.add(GRU(50, return_sequences=True, input_shape=(sequence_length, 1)))
gru_model.add(GRU(50))
gru_model.add(Dense(1))
gru_model.compile(optimizer='adam', loss='mean_squared_error')



# Train the GRU model
gru_model.fit(X_train, y_train, epochs=10, batch_size=32)

#predictions
gru_train_predictions = gru_model.predict(X_train)
gru_test_predictions = gru_model.predict(X_test)

# Scale back the predictions to the original range
gru_train_predictions = scaler.inverse_transform(gru_train_predictions)
gru_test_predictions = scaler.inverse_transform(gru_test_predictions)

#flatten
gru_train_predictions = [x[0] for x in gru_train_predictions]
gru_test_predictions = [x[0] for x in gru_test_predictions]


2023-06-04 12:19:51.743918: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 12:19:51.745806: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 12:19:51.747291: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/10


2023-06-04 12:19:51.964776: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 12:19:51.967497: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 12:19:51.970531: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2/2 [==============================] - 4s 6ms/step - loss: 0.0802
Epoch 2/10
2/2 [==============================] - 0s 5ms/step - loss: 0.0717
Epoch 3/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0624
Epoch 4/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0548
Epoch 5/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0491
Epoch 6/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0437
Epoch 7/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0406
Epoch 8/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0385
Epoch 9/10
2/2 [==============================] - 0s 8ms/step - loss: 0.0373
Epoch 10/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0381


2023-06-04 12:19:56.543190: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 12:19:56.544834: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 12:19:56.546527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 23ms/step


In [81]:
import plotly.graph_objects as go

# Assuming you have the training values in train_data, testing values in test_data,
# in-sample predictions in in_sample_predictions, and holdout predictions in holdout_predictions

# Create a trace for the training values
trace_train = go.Scatter(
    x=train_data.index,
    y=train_data['transaction_amount'],
    mode='lines',
    name='Training'
)

# Create a trace for the testing values
trace_test = go.Scatter(
    x=test_data.index,
    y=test_data['transaction_amount'],
    mode='lines',
    name='Testing'
)

# Create a trace for the in-sample predictions
trace_insample = go.Scatter(
    x=train_data.index,
    y=gru_train_predictions,
    mode='lines',
    name='In-Sample Predictions'
)

# Create a trace for the holdout predictions
trace_holdout = go.Scatter(
    x=test_data.index,
    y=gru_test_predictions,
    mode='lines',
    name='Holdout Predictions'
)

# Create the data list for the plot
data = [trace_train, trace_test, trace_insample, trace_holdout]

# Create the layout for the plot
layout = go.Layout(
    title='GRU Training, Testing, In-Sample, and Holdout Predictions',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Transaction Amount')
)

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Show the figure
fig.show()


In [88]:
from sklearn.metrics import mean_squared_error

# Calculate mean squared error
mse = mean_squared_error(y_test, gru_test_predictions)

# Print the mean squared error
print("Mean Squared Error:", mse)

Mean Squared Error: 1939484.2918532153


In [78]:
# Build the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=True, input_shape=(sequence_length, 1)))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions
lstm_train_predictions = lstm_model.predict(X_train)
lstm_test_predictions = lstm_model.predict(X_test)

# Scale back the predictions to the original range

lstm_train_predictions = scaler.inverse_transform(lstm_train_predictions)
lstm_test_predictions = scaler.inverse_transform(lstm_test_predictions)

lstm_train_predictions = [x[0] for x in lstm_train_predictions]
lstm_test_predictions = [x[0] for x in lstm_test_predictions]

2023-06-04 12:26:18.349868: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 12:26:18.352032: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 12:26:18.353453: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/10


2023-06-04 12:26:18.619765: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 12:26:18.622129: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 12:26:18.623623: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2/2 [==============================] - 4s 5ms/step - loss: 0.0859
Epoch 2/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0829
Epoch 3/10
2/2 [==============================] - 0s 8ms/step - loss: 0.0798
Epoch 4/10
2/2 [==============================] - 0s 9ms/step - loss: 0.0766
Epoch 5/10
2/2 [==============================] - 0s 11ms/step - loss: 0.0737
Epoch 6/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0705
Epoch 7/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0680
Epoch 8/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0649
Epoch 9/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0623
Epoch 10/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0594


2023-06-04 12:26:22.925198: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-04 12:26:22.926797: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-04 12:26:22.928853: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 24ms/step


In [82]:
import plotly.graph_objects as go

# Assuming you have the training values in train_data, testing values in test_data,
# in-sample predictions in in_sample_predictions, and holdout predictions in holdout_predictions

# Create a trace for the training values
trace_train = go.Scatter(
    x=train_data.index,
    y=train_data['transaction_amount'],
    mode='lines',
    name='Training'
)

# Create a trace for the testing values
trace_test = go.Scatter(
    x=test_data.index,
    y=test_data['transaction_amount'],
    mode='lines',
    name='Testing'
)

# Create a trace for the in-sample predictions
trace_insample = go.Scatter(
    x=train_data.index,
    y=lstm_train_predictions,
    mode='lines',
    name='In-Sample Predictions'
)

# Create a trace for the holdout predictions
trace_holdout = go.Scatter(
    x=test_data.index,
    y=lstm_test_predictions,
    mode='lines',
    name='Holdout Predictions'
)

# Create the data list for the plot
data = [trace_train, trace_test, trace_insample, trace_holdout]

# Create the layout for the plot
layout = go.Layout(
    title='LSTM Training, Testing, In-Sample, and Holdout Predictions',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Transaction Amount')
)

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Show the figure
fig.show()


In [87]:
from sklearn.metrics import mean_squared_error

y_test = scaler.inverse_transform(y_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, lstm_test_predictions)

# Print the mean squared error
print("Mean Squared Error:", mse)

Mean Squared Error: 3191721.8291636794


In [85]:
lstm_test_predictions

[674.4687, 638.6857, 692.9822]

In [86]:
y_test

array([[0.01845596],
       [0.2471104 ],
       [0.74052309]])